In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/sales_data_sample.csv', encoding='unicode_escape')
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [4]:
# Inspecting data types
df.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')

In [ ]:
# Converting to datetime and fix format
df['ORDERDATE'] = pd.to_datetime(df.ORDERDATE, format='%m/%d/%Y %H:%M')
df['ORDERDATE'] = df.ORDERDATE.dt.strftime('%Y-%m-%d %H:%M')
df.head()

In [ ]:
df['ORDERDATE'] = pd.to_datetime(df.ORDERDATE)
df.tail()

## Working dimension Time

In [ ]:
df_time = df[['ORDERDATE', 'MONTH_ID', 'QTR_ID', 'YEAR_ID']].copy(deep=True)
df_time.head()

## First step will be getting derivate values

### At this moment the dimension time should have the following fields:
- dateid:: yyyymmdd
- day:: dd
- dayname:: (monday-tuesday- and so on)
- month:: mm
- monthname:: (January-February- and so on)
- is_day_of_week:: True/False
- MonthNumberByYear:: 2023-02
- Weeknumber
- Semester
- Quarter
- Year

In [ ]:
# Create dateid
df_time['DATEID'] = df_time.ORDERDATE.dt.strftime('%Y%m%d')
df_time['DAY']    = df_time.ORDERDATE.dt.day
df_time['DAYNAME']    = df_time.ORDERDATE.dt.day_name()
df_time['MONTHNAME']  = df_time.ORDERDATE.dt.month_name()
df_time['WEEKNUMBER']  = df_time.ORDERDATE.dt.isocalendar().week
df_time['IS_WEEKDAY'] = df_time['ORDERDATE'].apply(lambda x: 1 if x.weekday() else 0)
df_time['SEMESTER'] = df_time['MONTH_ID'].apply(lambda x: 1 if x < 7 else 2)
df_time['MONTHNUMBERBYYEAR'] = df_time.ORDERDATE.dt.strftime('%Y-%m')
df_time.rename(columns={"ORDERDATE": "FULLDATE", 'YEAR_ID': 'YEAR', 'QTR_ID': 'QUARTER', 'MONTH_ID': 'MONTH'}, inplace=True)
df_time.head()

In [ ]:
# Reorder dataframe
cols = ['DATEID', 'YEAR', 'SEMESTER', 'QUARTER', 'MONTHNAME', 'MONTH', 'MONTHNUMBERBYYEAR', 'WEEKNUMBER', 'DAYNAME', 'DAY', 'IS_WEEKDAY', 'FULLDATE']
df_time = df_time.reindex(columns=cols)
df_time.head()

In [ ]:
print(f'Dimension Date shape: {df_time.shape}')

In [ ]:
df_time.drop_duplicates(keep='first',inplace=True)
print(f'Dimension Date shape after dedup: {df_time.shape}')

In [ ]:
df_time.sort_values(by=['FULLDATE'], inplace=True)
df_time.to_csv('output_files/dim_date.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('datasets/sales_data_sample.csv', encoding='unicode_escape')

# Converting to datetime and fix format
df['ORDERDATE'] = pd.to_datetime(df.ORDERDATE, format='%m/%d/%Y %H:%M')
df['ORDERDATE'] = df.ORDERDATE.dt.strftime('%Y-%m-%d %H:%M')

df_time = df[['ORDERDATE', 'MONTH_ID', 'QTR_ID', 'YEAR_ID']].copy(deep=True)

df_time['ORDERDATE'] = pd.to_datetime(df_time.ORDERDATE)

# Create derivative columns
df_time['DATEID'] = df_time.ORDERDATE.dt.strftime('%Y%m%d')
df_time['DAY']    = df_time.ORDERDATE.dt.day
df_time['DAYNAME']    = df_time.ORDERDATE.dt.day_name()
df_time['MONTHNAME']  = df_time.ORDERDATE.dt.month_name()
df_time['WEEKNUMBER']  = df_time.ORDERDATE.dt.isocalendar().week
df_time['IS_WEEKDAY'] = df_time['ORDERDATE'].apply(lambda x: 1 if x.weekday() else 0)
df_time['SEMESTER'] = df_time['MONTH_ID'].apply(lambda x: 1 if x < 7 else 2)
df_time['MONTHNUMBERBYYEAR'] = df_time.ORDERDATE.dt.strftime('%Y-%m')

# Renaming columns
df_time.rename(columns={"ORDERDATE": "FULLDATE", 'YEAR_ID': 'YEAR',
                        'QTR_ID': 'QUARTER', 'MONTH_ID': 'MONTH'}, inplace=True)

# Reorder dataframe
cols = ['DATEID', 'YEAR', 'SEMESTER', 'QUARTER', 'MONTHNAME', 'MONTH', 'MONTHNUMBERBYYEAR',
        'WEEKNUMBER', 'DAYNAME', 'DAY', 'IS_WEEKDAY', 'FULLDATE']
df_time = df_time.reindex(columns=cols)

# Drop duplicates
df_time.drop_duplicates(keep='first',inplace=True)

# Sorting dataframe
df_time.sort_values(by=['FULLDATE'], inplace=True)

# Exporting Results
df_time.to_csv('output_files/dim_date.csv', index=False)

## Working dimension Location/geography

In [ ]:
df.head()

In [ ]:
df_geo = df[['CITY', 'STATE', 'COUNTRY', 'TERRITORY']].copy(deep=True)
df_geo.head()

In [ ]:
print(f'Dimension Location shape: {df_geo.shape}')

In [ ]:
df_geo.drop_duplicates(keep='first',inplace=True)
print(f'Dimension Location shape after dedup: {df_geo.shape}')
df_geo.to_csv('output_files/dim_location.csv', index=False)

## Working dimension Products

In [ ]:
df_prod = df[['PRODUCTCODE', 'PRODUCTLINE', 'MSRP']].copy(deep=True)
df_prod.head()

In [ ]:
print(f'Dimension Product shape: {df_prod.shape}')

In [ ]:
df_prod.drop_duplicates(keep='first',inplace=True)
df_prod.reset_index(drop=True, inplace=True)
df_prod.index += 1

In [ ]:
df_prod.reset_index(inplace=True)

In [ ]:
df_prod.head()

In [ ]:
df_prod.columns = ['PRODUCTID', 'PRODUCTCODE', 'PRODUCTLINE', 'MSRP']
print(f'Dimension Product shape after dedup: {df_prod.shape}')
df_prod.to_csv('output_files/dim_product.csv', index=False)

## Working dimension Customer

In [ ]:
df.head()

In [ ]:
df_customer = df[['CUSTOMERNAME', 'PHONE', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE', 'COUNTRY', 'TERRITORY', ]].copy(deep=True)
df_customer.head()

In [ ]:
print(f'Dimension Customer shape: {df_customer.shape}')

In [ ]:
df_customer.drop_duplicates(keep='first',inplace=True)
df_customer.reset_index(drop=True, inplace=True)
df_customer.index += 1

In [ ]:
df_customer.reset_index(inplace=True)
# df_customer['LOCATIONID'] = df_customer['index']

In [ ]:
df_customer.columns

In [ ]:
df_customer = df_customer[['index', 'CUSTOMERNAME', 'PHONE', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME',
                           'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
                           'COUNTRY', 'TERRITORY']]
df_customer.rename(columns={'index': 'CUSTOMERID'}, inplace=True)
df_customer.head()

In [ ]:
df_customer.drop_duplicates(keep='first',inplace=True)
print(f'Dimension Customer shape after dedup: {df_customer.shape}')
df_customer.to_csv('output_files/dim_customer.csv', index=False)

## Work in factorderlines

In [ ]:
df.head()

In [ ]:
df[df.ORDERNUMBER == 10145]

In [ ]:
df_factorderlines = df[['ORDERNUMBER', 'ORDERLINENUMBER', 'ORDERDATE', 'CUSTOMERNAME', 'PRODUCTCODE', 'QUANTITYORDERED', 'PRICEEACH', 'SALES','DEALSIZE']].copy(deep=True)
df_factorderlines.reset_index(drop=True, inplace=True)
df_factorderlines.head()

In [ ]:
# Add CustomerId
df_merged = df_factorderlines.merge(df_customer[['CUSTOMERNAME', 'CUSTOMERID']], on='CUSTOMERNAME', how='left')
df_merged.head()

In [ ]:
# Add DateID
df_merged = df_merged.merge(df_time[['FULLDATE', 'DATEID']], left_on='ORDERDATE', right_on='FULLDATE', how='left')
df_merged.head()

In [ ]:
# Add ProductID
df_merged = df_merged.merge(df_prod[['PRODUCTCODE', 'PRODUCTID']], on='PRODUCTCODE', how='left')
df_merged.head()

In [ ]:
# Drop ORDERDATE, FULLDATE, CUSTOMERNAME, AND PRODUCTCODE
df_merged.drop(columns=['ORDERDATE', 'FULLDATE', 'CUSTOMERNAME', 'PRODUCTCODE'], inplace=True)
df_merged.head()

In [ ]:
df_merged = df_merged[['PRODUCTID', 'CUSTOMERID', 'DATEID', 'ORDERNUMBER', 'ORDERLINENUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'SALES','DEALSIZE']]
df_merged['TOTAL_ORDERLINE_AMOUNT'] = df_merged.apply(lambda row: round(row['PRICEEACH'] * row['QUANTITYORDERED'],2), axis=1)
df_merged.head()

In [ ]:
df_merged = df_merged[['PRODUCTID', 'CUSTOMERID', 'DATEID', 'ORDERNUMBER', 'ORDERLINENUMBER', 'DEALSIZE', 'QUANTITYORDERED', 'PRICEEACH', 'SALES', 'TOTAL_ORDERLINE_AMOUNT']]
df_merged[df_merged['SALES'].round(2) != df_merged['TOTAL_ORDERLINE_AMOUNT'].round(2)].head()

In [ ]:
df_merged.drop_duplicates(keep='first', inplace=True)
df_merged.to_csv('output_files/factorderlines.csv', index=False)

## Work in factorders

In [ ]:
df_factorders = df[['CUSTOMERNAME', 'ORDERDATE', 'ORDERNUMBER', 'ORDERLINENUMBER', 'STATUS']].copy(deep=True)
df_factorders.head()

In [ ]:
df_join = df_factorders.merge(df_merged[['ORDERNUMBER', 'ORDERLINENUMBER', 'CUSTOMERID', 'DATEID', 'SALES']], on=['ORDERNUMBER', 'ORDERLINENUMBER'], how='left')
df_join.head()

In [ ]:
# Drop columns 'CUSTOMERNAME', AND 'ORDERDATE'
df_join.drop(columns=['CUSTOMERNAME', 'ORDERDATE'], inplace=True)
df_join.head()

In [ ]:
factorders = df_join.groupby(['ORDERNUMBER', 'DATEID', 'CUSTOMERID', 'STATUS'])[['SALES']].sum()

In [ ]:
factorders.head()

In [ ]:
factorders.reset_index(inplace=True)
factorders = factorders[['CUSTOMERID', 'DATEID','ORDERNUMBER', 'STATUS', 'SALES']]

In [ ]:
factorders.rename(columns={'SALES': 'TOTAL_ORDER_AMOUNT'}, inplace=True)
factorders.head()

In [ ]:
factorders.to_csv('output_files/factorders.csv', index=False)

In [ ]:
df_customer.head()

In [ ]:
len(df_customer.columns)